<a href="https://colab.research.google.com/github/umak1106/Sumoylation/blob/main/RSCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from itertools import cycle
import numpy as np
import matplotlib.pyplot as plt
import math
import os, re, sys
import pandas as pd
from collections import Counter

# Save forecast results
def save_predict_result(data, output):
    with open(output, 'w') as f:
        if len(data)>1:
            for i in range(len(data)):
                f.write('# result for fold %d\n' % (i + 1))
                for j in range(len(data[i])):
                    f.write('%d\t%s\n' % (data[i][j][0], data[i][j][2]))
        else:
            for i in range(len(data)):
                f.write('# result for predict\n')
                for j in range(len(data[i])):
                    f.write('%d\t%s\n' % (data[i][j][0], data[i][j][2]))
        f.close()
    return None

# Plot the ROC curve and return the AUC value
def plot_roc_curve(data, output, label_column=0, score_column=2):
    datasize = len(data)
    tprs = []
    aucs = []
    fprArray = []
    tprArray = []
    thresholdsArray = []
    mean_fpr = np.linspace(0, 1, 100)
    for i in range(len(data)):
        fpr, tpr, thresholds = roc_curve(data[i][:, label_column], data[i][:, score_column])
        fprArray.append(fpr)
        tprArray.append(tpr)
        thresholdsArray.append(thresholds)
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'blueviolet', 'deeppink'])
    fig = plt.figure(figsize=(7,7))
    for i, color in zip(range(len(fprArray)), colors):
        if datasize>1:
            plt.plot(fprArray[i], tprArray[i], lw=1, alpha=0.7, color=color,
                     label='ROC fold %d (AUC = %0.4f)' % (i + 1, aucs[i]))
        else:
            plt.plot(fprArray[i], tprArray[i], lw=1, alpha=0.7, color=color,
                     label='ROC (AUC = %0.4f)' %aucs[i])
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
             label='Random', alpha=.8)
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    # Calculate the standard deviation
    std_auc = np.std(aucs)
    if datasize>1:
        plt.plot(mean_fpr, mean_tpr, color='blue',
                 label=r'Mean ROC (AUC = %0.4f $\pm$ %0.3f)' % (mean_auc, std_auc),
                 lw=2, alpha=.9)
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    if datasize>1:
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')
    plt.xlim([0, 1.0])
    plt.ylim([0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.savefig(output)
    plt.close(0)
    return mean_auc, aucs

# Plot the PRC curve and return the PRC value
def plot_prc_curve(data, output, label_column=0, score_column=2):
    datasize = len(data)
    precisions = []
    aucs = []
    recall_array = []
    precision_array = []
    mean_recall = np.linspace(0, 1, 100)

    for i in range(len(data)):
        precision, recall, _ = precision_recall_curve(data[i][:, label_column], data[i][:, score_column])
        recall_array.append(recall)
        precision_array.append(precision)
        precisions.append(np.interp(mean_recall, recall[::-1], precision[::-1])[::-1])
        roc_auc = auc(recall, precision)
        aucs.append(roc_auc)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'blueviolet', 'deeppink', 'cyan'])
    # ROC plot for CV
    fig = plt.figure(figsize=(7,7))
    for i, color in zip(range(len(recall_array)), colors):
        if datasize>1:
            plt.plot(recall_array[i], precision_array[i], lw=1, alpha=0.7, color=color,
                     label='PRC fold %d (AUPRC = %0.4f)' % (i + 1, aucs[i]))
        else:
            plt.plot(recall_array[i], precision_array[i], lw=1, alpha=0.7, color=color,
                     label='PRC (AUPRC = %0.4f)' %aucs[i])
    mean_precision = np.mean(precisions, axis=0)
    mean_recall = mean_recall[::-1]
    mean_auc = auc(mean_recall, mean_precision)
    std_auc = np.std(aucs)

    if datasize>1:
        plt.plot(mean_recall, mean_precision, color='blue',
                 label=r'Mean PRC (AUPRC = %0.4f $\pm$ %0.3f)' % (mean_auc, std_auc),
                 lw=2, alpha=.9)
    std_precision = np.std(precisions, axis=0)
    precision_upper = np.minimum(mean_precision + std_precision, 1)
    precision_lower = np.maximum(mean_precision - std_precision, 0)
    if datasize>1:
        plt.fill_between(mean_recall, precision_lower, precision_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')
    plt.xlim([0, 1.0])
    plt.ylim([0, 1.0])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend(loc="lower left")
    plt.savefig(output)
    plt.close(0)
    return mean_auc, aucs

# Calculate and save performance metrics
def calculate_metrics(labels, scores, cutoff=0.5, po_label=1):
    my_metrics = {
        'SN': 'NA',
        'SP': 'NA',
        'ACC': 'NA',
        'MCC': 'NA',
        'Recall': 'NA',
        'Precision': 'NA',
        'F1-score': 'NA',
        'Cutoff': cutoff,
    }

    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(scores)):
        if labels[i] == po_label:
            if scores[i] >= cutoff:
                tp = tp + 1
            else:
                fn = fn + 1
        else:
            if scores[i] < cutoff:
                tn = tn + 1
            else:
                fp = fp + 1

    my_metrics['SN'] = tp / (tp + fn) if (tp + fn) != 0 else 'NA'
    my_metrics['SP'] = tn / (fp + tn) if (fp + tn) != 0 else 'NA'
    my_metrics['ACC'] = (tp + tn) / (tp + fn + tn + fp)
    my_metrics['MCC'] = (tp * tn - fp * fn) / np.math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) if (
                                                                                                                     tp + fp) * (
                                                                                                                     tp + fn) * (
                                                                                                                     tn + fp) * (
                                                                                                                     tn + fn) != 0 else 'NA'
    my_metrics['Precision'] = tp / (tp + fp) if (tp + fp) != 0 else 'NA'
    my_metrics['Recall'] = my_metrics['SN']
    my_metrics['F1-score'] = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 'NA'
    return my_metrics

def calculate_metrics_list(data, label_column=0, score_column=2, cutoff=0.5, po_label=1):
    metrics_list = []
    for i in data:
        metrics_list.append(calculate_metrics(i[:, label_column], i[:, score_column], cutoff=cutoff, po_label=po_label))
    if len(metrics_list) == 1:
        return metrics_list
    else:
        mean_dict = {}
        std_dict = {}
        keys = metrics_list[0].keys()
        for i in keys:
            mean_list = []
            for metric in metrics_list:
                mean_list.append(metric[i])
            mean_dict[i] = np.array(mean_list).sum() / len(metrics_list)
            std_dict[i] = np.array(mean_list).std()
        metrics_list.append(mean_dict)
        metrics_list.append(std_dict)
        return metrics_list

def save_prediction_metrics_list(metrics_list, output):
    if len(metrics_list) == 1:
        with open(output, 'w') as f:
            f.write('Result')
            for keys in metrics_list[0]:
                f.write('\t%s' % keys)
            f.write('\n')
            for i in range(len(metrics_list)):
                f.write('value')
                for keys in metrics_list[i]:
                    f.write('\t%s' % metrics_list[i][keys])
                f.write('\n')
            f.close()
    else:
        with open(output, 'w') as f:
            f.write('Fold')
            for keys in metrics_list[0]:
                f.write('\t%s' % keys)
            f.write('\n')
            for i in range(len(metrics_list)):
                if i <= len(metrics_list)-3:
                    f.write('%d' % (i + 1))
                elif i == len(metrics_list)-2:
                    f.write('mean')
                else:
                    f.write('std')
                for keys in metrics_list[i]:
                    f.write('\t%s' % metrics_list[i][keys])
                f.write('\n')
            f.close()
    return None

# Fixed SP value, computing performance
def fixed_sp_calculate_metrics_list(data, cutoffs, label_column=0, score_column=1, po_label=1):
    metrics_list = []
    for index,i in enumerate(data):
        metrics_list.append(calculate_metrics(i[:, label_column], i[:, score_column], cutoff=cutoffs[index], po_label=po_label))
    if len(metrics_list) == 1:
        return metrics_list
    else:
        mean_dict = {}
        std_dict = {}
        keys = metrics_list[0].keys()
        for i in keys:
            mean_list = []
            for metric in metrics_list:
                mean_list.append(metric[i])
            mean_dict[i] = np.array(mean_list).sum() / len(metrics_list)
            std_dict[i] = np.array(mean_list).std()
        metrics_list.append(mean_dict)
        metrics_list.append(std_dict)
        return metrics_list

In [7]:
import pandas as pd
import os,sys,re
import numpy as np
import itertools
from collections import Counter
from numpy import *

def Binary(filepath, CodeType):
    AA = 'ACDEFGHIKLMNPQRSTVWYX'
    encodings = []
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = []
        for aa in sequence:
            singlecode = []
            for aa1 in AA:
                tag = 1 if aa == aa1 else 0
                singlecode.append(tag)
            code.append(singlecode)
        encodings.append(code)
    if CodeType == 1:
        return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)
    else:
        return np.array(encodings).astype(np.float64).reshape(len(encodings),-1), np.array(label).astype(np.float64)

def EAAC(filepath, CodeType, windows=5):
    AA = 'ACDEFGHIKLMNPQRSTVWY'
    encodings = []
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = []
        for j in range(len(sequence)):
            singlecode = []
            if j < len(sequence) and j + windows <= len(sequence):
                count = Counter(re.sub('X', '', sequence[j:j + windows]))
                for key in count:
                    count[key] = count[key] / windows
                for aa in AA:
                    singlecode.append(count[aa])
                code.append(singlecode)
        encodings.append(code)
    if CodeType == 1:
        return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)
    else:
        return np.array(encodings).astype(np.float64).reshape(len(encodings),-1), np.array(label).astype(np.float64)

def Binary_of_bigram(filepath):
    aaPairs = []
    encodings = []
    for i in itertools.product('ACDEFGHIKLMNPQRSTVWY', repeat=2):
        aaPairs.append(''.join(i))
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = []
        for index in range(len(sequence)-1):
            singercode = np.zeros(len(aaPairs))
            pattern = '' + sequence[index] + sequence[index+1]
            if pattern in aaPairs:
            	    singercode[aaPairs.index(pattern)] = 1
            code.append(singercode)
        encodings.append(code)
    return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)

def BLOSUM62(filepath,CodeType):
    blosum62 = {
        'A': [4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1, -1, -2, -1, 1, 0, -3, -2, 0],  # A
        'R': [-1, 5, 0, -2, -3, 1, 0, -2, 0, -3, -2, 2, -1, -3, -2, -1, -1, -3, -2, -3],  # R
        'N': [-2, 0, 6, 1, -3, 0, 0, 0, 1, -3, -3, 0, -2, -3, -2, 1, 0, -4, -2, -3],  # N
        'D': [-2, -2, 1, 6, -3, 0, 2, -1, -1, -3, -4, -1, -3, -3, -1, 0, -1, -4, -3, -3],  # D
        'C': [0, -3, -3, -3, 9, -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1],  # C
        'Q': [-1, 1, 0, 0, -3, 5, 2, -2, 0, -3, -2, 1, 0, -3, -1, 0, -1, -2, -1, -2],  # Q
        'E': [-1, 0, 0, 2, -4, 2, 5, -2, 0, -3, -3, 1, -2, -3, -1, 0, -1, -3, -2, -2],  # E
        'G': [0, -2, 0, -1, -3, -2, -2, 6, -2, -4, -4, -2, -3, -3, -2, 0, -2, -2, -3, -3],  # G
        'H': [-2, 0, 1, -1, -3, 0, 0, -2, 8, -3, -3, -1, -2, -1, -2, -1, -2, -2, 2, -3],  # H
        'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4, 2, -3, 1, 0, -3, -2, -1, -3, -1, 3],  # I
        'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2, 4, -2, 2, 0, -3, -2, -1, -2, -1, 1],  # L
        'K': [-1, 2, 0, -1, -3, 1, 1, -2, -1, -3, -2, 5, -1, -3, -1, 0, -1, -3, -2, -2],  # K
        'M': [-1, -1, -2, -3, -1, 0, -2, -3, -2, 1, 2, -1, 5, 0, -2, -1, -1, -1, -1, 1],  # M
        'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0, 0, -3, 0, 6, -4, -2, -2, 1, 3, -1],  # F
        'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7, -1, -1, -4, -3, -2],  # P
        'S': [1, -1, 1, 0, -1, 0, 0, 0, -1, -2, -2, 0, -1, -2, -1, 4, 1, -3, -2, -2],  # S
        'T': [0, -1, 0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1, 5, -2, -2, 0],  # T
        'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1, -4, -3, -2, 11, 2, -3],  # W
        'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2, -1, -1, -2, -1, 3, -3, -2, -2, 2, 7, -1],  # Y
        'V': [0, -3, -3, -3, -1, -2, -2, -3, -3, 3, 1, -2, 1, -1, -2, -2, 0, -3, -1, 4],  # V
        'X': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # X
    }
    encodings = []
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = []
        for aa in sequence:
            singlecode = []
            if aa in blosum62.keys():
                singlecode = singlecode + blosum62[aa]
            else:
                singlecode = singlecode + blosum62['-']
            code.append(singlecode)
        encodings.append(code)
    if CodeType == 1:
        return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)
    else:
        return np.array(encodings).astype(np.float64).reshape(len(encodings),-1), np.array(label).astype(np.float64)

def ZScale(filepath, CodeType):
    zscale = {
        'A': [0.24, -2.32, 0.60, -0.14, 1.30],  # A
        'C': [0.84, -1.67, 3.71, 0.18, -2.65],  # C
        'D': [3.98, 0.93, 1.93, -2.46, 0.75],  # D
        'E': [3.11, 0.26, -0.11, -0.34, -0.25],  # E
        'F': [-4.22, 1.94, 1.06, 0.54, -0.62],  # F
        'G': [2.05, -4.06, 0.36, -0.82, -0.38],  # G
        'H': [2.47, 1.95, 0.26, 3.90, 0.09],  # H
        'I': [-3.89, -1.73, -1.71, -0.84, 0.26],  # I
        'K': [2.29, 0.89, -2.49, 1.49, 0.31],  # K
        'L': [-4.28, -1.30, -1.49, -0.72, 0.84],  # L
        'M': [-2.85, -0.22, 0.47, 1.94, -0.98],  # M
        'N': [3.05, 1.62, 1.04, -1.15, 1.61],  # N
        'P': [-1.66, 0.27, 1.84, 0.70, 2.00],  # P
        'Q': [1.75, 0.50, -1.44, -1.34, 0.66],  # Q
        'R': [3.52, 2.50, -3.50, 1.99, -0.17],  # R
        'S': [2.39, -1.07, 1.15, -1.39, 0.67],  # S
        'T': [0.75, -2.18, -1.12, -1.46, -0.40],  # T
        'V': [-2.59, -2.64, -1.54, -0.85, -0.02],  # V
        'W': [-4.36, 3.94, 0.59, 3.44, -1.59],  # W
        'Y': [-2.54, 2.44, 0.43, 0.04, -1.47],  # Y
        'X': [0.00, 0.00, 0.00, 0.00, 0.00],  # X
    }
    encodings = []
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = []
        for aa in sequence:
            singlecode = []
            if aa in zscale.keys():
                singlecode = singlecode + zscale[aa]
            else:
                singlecode = singlecode + zscale['-']
            code.append(singlecode)
        encodings.append(code)
    if CodeType == 1:
        return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)
    else:
        return np.array(encodings).astype(np.float64).reshape(len(encodings),-1), np.array(label).astype(np.float64)

def EGAAC(filepath, CodeType, window = 5):
    group = {
        'alphaticr': 'GAVLMI',
        'aromatic': 'FYW',
        'postivecharger': 'KRH',
        'negativecharger': 'DE',
        'uncharger': 'STCPNQ'
    }
    groupKey = group.keys()
    encodings = []
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = []
        for j in range(len(sequence)):
            if j + window <= len(sequence):
                singlecode = []
                count = Counter(re.sub('X', '', sequence[j:j + window]))
                myDict = {}
                for key in groupKey:
                    for aa in group[key]:
                        myDict[key] = myDict.get(key, 0) + count[aa]
                for key in groupKey:
                    singlecode.append(myDict[key] / window)
                code.append(singlecode)
        encodings.append(code)
    if CodeType == 1:
        return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)
    else:
        return np.array(encodings).astype(np.float64).reshape(len(encodings),-1), np.array(label).astype(np.float64)

def getPSSM(filepath, CodeType):
    encodings = []
    label = []
    files = os.listdir(filepath) # 读入文件夹
    sorted_files = sorted(files,key = lambda i:int(re.findall(r'_(\d+)-',i)[0]))
    for file in sorted_files:
        label.append(int(file[file.find('.')-1]))
        pssm = []
        with open(os.path.join(filepath, file)) as f:
            lines = f.readlines()[3:38]
            pssm = np.array([line.split()[2:22] for line in lines], dtype=int)
            f.close()
            encodings.append(pssm)
    if CodeType == 1:
        return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)
    else:
        return np.array(encodings).astype(np.float64).reshape(len(encodings),-1), np.array(label).astype(np.float64)


def AAindex(filepath, CodeType):
    AA = 'ARNDCQEGHILKMFPSTWYV'
    fileAAindex = r'D:\jupyter\DeepSUMO-master\Utils\data\Sort_AAindex.txt'
    with open(fileAAindex) as f:
        records = f.readlines()[1:15]
    AAindex = []
    AAindexName = []
    for i in records:
        AAindex.append(i.rstrip().split()[1:] if i.rstrip() != '' else None)
        AAindexName.append(i.rstrip().split()[0] if i.rstrip() != '' else None)
    encodings = []
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = []
        for j in sequence:
            single_code = []
            for k in AAindexName:
                if j in AA:
                    value = AAindex[AAindexName.index(k)][AA.index(j)]
                else:
                    value = 0
                single_code.append(value)
            code.append(single_code)
        encodings.append(code)
    if CodeType == 1:
        return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)
    else:
        return np.array(encodings).astype(np.float64).reshape(len(encodings),-1), np.array(label).astype(np.float64)

def CKSAAP(filepath, gap = 0):
    AA = 'ACDEFGHIKLMNPQRSTVWY'
    aaPairs = []
    patten = re.compile('X|U|-|_')
    for aa1 in AA:
        for aa2 in AA:
            aaPairs.append(aa1 + aa2)
    encodings = []
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = []
        for g in range(gap + 1):
            myDict = {}
            for pair in aaPairs:
                myDict[pair] = 0
            sum = 0
            for index1 in range(len(sequence)):
                index2 = index1 + g + 1
                if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[
                    index2] in AA:
                    myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
                    sum = sum + 1
            for pair in aaPairs:
                code.append(myDict[pair] / sum)
        encodings.append(code)
    return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)

# Construct four sets of features (0, 1 notation) based on the properties of amino acids that appear at -2, -1 and +1, +2
def Statistics_property(filepath):
    dataframe = pd.read_csv(filepath)
    label = list(dataframe['Label'])
    sequences = list(dataframe['Sequence'])
    encodings = []
    middle = len(sequences[0])//2
    positive_cahrge = 'DE'
    negative_charge = 'RKH'
    charge = 'DERKH'
    Hydrophobic = 'AFGILPVWY'
    type1 = 'ILV'
    type2 = 'AFMPW'
    type3 = 'GY'

    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        code = zeros(11)
        if sequence[middle-2] in positive_cahrge:
            code[0] = 1
        else:
            if sequence[middle-2] in negative_charge:
                code[1] = 1
        if sequence[middle-1] in type1:
            code[5] = 1
        else:
            if sequence[middle-1] in type2:
                code[4] = 1
            else:
                if sequence[middle-1] in type3:
                    code[3] = 1
                else:
                    code[2] = 1
        if sequence[middle-1] in charge:
            code[6] = 1
        if sequence[middle+1] in Hydrophobic:
            code[7] = 1
        if sequence[middle+1] in charge:
            code[8] = 1
        if sequence[middle+2] in positive_cahrge:
            code[9] = 1
        else:
            if sequence[middle+2] in negative_charge:
                code[10] = 1

        encodings.append(code)
    return np.array(encodings).astype(np.float64), np.array(label).astype(np.float64)

def getSequence(train_fastas):
    pos_list = []
    neg_list = []
    sequences = list(train_fastas['Sequence'])
    labels = list(train_fastas['Label'])
    for s, l in zip(sequences, labels):
        if l==1:
            pos_list.append(s)
        else:
            neg_list.append(s)
    return pos_list, neg_list

# Determine whether it is a natural amino acid, use 'X' to represent all other amino acids, and construct a frequency matrix
def replace_no_native_amino_acid(lists):
    native_amino_acid = ('A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y',)
    frequency_array = zeros((21, len(lists[0])))
    flag = 1
    for site in range(len(lists)):
        for i in range(len(lists[0])):
            for j in range(len(native_amino_acid)):
                if lists[site][i] == native_amino_acid[j]:
                    frequency_array[j][i] = frequency_array[j][i] + 1
                    flag = 0
                    break
            if flag != 0:
                frequency_array[20][i] = frequency_array[20][i]+1
            flag = 1

    length = len(lists)
    for i in range(len(frequency_array)):
        for j in range(len(frequency_array[0])):
            frequency_array[i][j] =frequency_array[i][j]/length

    return frequency_array

# Subtract the negative sample frequency from the positive sample frequency to get the overall sample frequency
def result_frequency_matrix(positive_matrix, negative_matrix):
    result_matrix = zeros((len(positive_matrix), len(positive_matrix[0])))
    for i in range(len(positive_matrix)):
        for j in range(len(positive_matrix[0])):
            result_matrix[i][j] = positive_matrix[i][j] - negative_matrix[i][j]
    return result_matrix


# Convert ordinal representation to frequency representation or entropy representation
def to_site(lists, frequency_array):
    full_frequency_array = []
    for i in range(len(lists)):
        position = int(lists[i])
        full_frequency_array.append(frequency_array[position-1][i])
    return full_frequency_array

#Convert amino acid sequence to ordinal representation
def number_encoding(peptide):
    native_amino_acid = ('A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y',)
    numlists = zeros(len(peptide))
    flag = 1
    for i in range(len(peptide)):
        for j in range(len(native_amino_acid)):
            if peptide[i] == native_amino_acid[j]:
                numlists[i] = j+1
                flag = 0
                break
        if flag != 0:
            numlists[i] = 21
        flag = 1

    return numlists

def PSAAP(trainpath, testpath = None):
    traindf = pd.read_csv(trainpath)

    psiteList, nsiteList = getSequence(traindf)
    positive_frequency_matrix = replace_no_native_amino_acid(psiteList)
    negative_frequency_matrix = replace_no_native_amino_acid(nsiteList)
    frequency_matrix = result_frequency_matrix(positive_frequency_matrix, negative_frequency_matrix)

    train_encodings = []
    train_label = list(traindf['Label'])
    sequences = list(traindf['Sequence'])
    for i in sequences:
        sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
        number_seq = number_encoding(sequence)
        code = to_site(number_seq, frequency_matrix)
        train_encodings.append(code)
    if testpath:
        testdf = pd.read_csv(testpath)
        test_encodings = []
        test_label = list(testdf['Label'])
        sequences1 = list(testdf['Sequence'])
        for i in sequences1:
            sequence = re.sub('[^ACDEFGHIKLMNPQRSTVWYX]', 'X', ''.join(i).upper())
            number_seq = number_encoding(sequence)
            code = to_site(number_seq, frequency_matrix)
            test_encodings.append(code)
        return np.array(train_encodings).astype(np.float64), np.array(train_label).astype(np.float64), np.array(test_encodings).astype(np.float64), np.array(test_label).astype(np.float64)

    return np.array(train_encodings).astype(np.float64), np.array(train_label).astype(np.float64)

In [3]:
import os,re
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential, losses, optimizers
import pandas as pd
from tensorflow.keras import backend as K
from collections import Counter
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from itertools import cycle
from tensorflow.keras.layers import Layer

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

def save_result(cv_res, ind_res, outPath, codename):
    mkdir(outPath)
    out = os.path.join(outPath, codename.lower())
    save_predict_result(cv_res, out + '_pre_cv.txt')
    cv_meanauc, cv_auc = plot_roc_curve(cv_res, out + '_roc_cv.png', label_column=0, score_column=2)
    cv_meanprc, cv_prc = plot_prc_curve(cv_res, out + '_prc_cv.png', label_column=0, score_column=2)
    cv_metrics = calculate_metrics_list(cv_res, label_column=0, score_column=2, cutoff=0.5, po_label=1)
    save_prediction_metrics_list(cv_metrics, out + '_metrics_cv.txt')
    save_predict_result(ind_res, out + '_pre_ind.txt')
    ind_meanauc, ind_auc = plot_roc_curve(ind_res, out + '_roc_ind.png', label_column=0, score_column=2)
    ind_meanprc, ind_prc = plot_prc_curve(ind_res, out + '_prc_ind.png', label_column=0, score_column=2)
    ind_metrics = calculate_metrics_list(ind_res, label_column=0, score_column=2, cutoff=0.5, po_label=1)
    save_prediction_metrics_list(ind_metrics, out + '_metrics_ind.txt')
    return None

# Create folder
def mkdir(path):
    path=path.strip()
    path=path.rstrip("\\")
    # Check if the path exists
    isExists=os.path.exists(path)
    if not isExists:
        # Create the directory if it doesn't exist
        os.makedirs(path)
    else:
        # Do not create directory if it exists
        pass

def res_net_block(input_data, filters, strides=1):
    x = layers.Conv1D(filters, kernel_size=3, strides=strides, padding='same')(input_data)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Conv1D(filters, kernel_size=3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    if strides != 1:
        downsample = layers.Conv1D(filters, kernel_size=1, strides=strides)(input_data)
    else:
        downsample = input_data
    x = layers.Add()([x, downsample])
    output = layers.Activation('relu')(x)

    return output

# CNN-based classifiers
def CNNbasic(Encode):
    inputs = tf.keras.Input(shape=(Encode.shape[1], Encode.shape[2]))

    x = layers.Conv1D(128, kernel_size=3)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool1D(pool_size=2, strides=1, padding='same')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Conv1D(128, kernel_size=3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool1D(pool_size=2, strides=1, padding='same')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation=tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dense(64, activation=tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    output = layers.Dense(1, activation=tf.nn.sigmoid)(x)
    DeepSUMO = Model(inputs=[inputs], outputs=[output], name="DeepSUMO")
    DeepSUMO.summary()
    DeepSUMO.compile(optimizer=optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'],
                  experimental_run_tf_function=False)
    return DeepSUMO

In [4]:


trainfilepath =  'five_fold_cross_validation.csv'
testfilepath = 'independent.csv'

In [9]:
ZScale1, y = ZScale(trainfilepath,1)
zscale_test, y_test = ZScale(testfilepath, 1)

In [12]:
print(ZScale1.shape)
print(zscale_test.shape)
print(y_test)


(67090, 39, 5)
(7456, 39, 5)
[1. 1. 1. ... 0. 0. 0.]


In [13]:
# The residual structure layered CNN architecture (RSCNN)
def RSCNN(Encode):

    inputs = tf.keras.Input(shape=(Encode.shape[1], Encode.shape[2]))
    x = layers.Conv1D(128, kernel_size=3)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool1D(pool_size=2, strides=1, padding='same')(x)
    x = layers.Dropout(0.5)(x)

    x = res_net_block(x, 128)
    x = layers.MaxPool1D(2)(x)
    x = layers.Dropout(0.5)(x)

    x = res_net_block(x, 128)
    x = layers.MaxPool1D(2)(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation=tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dense(64, activation=tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)

    output = layers.Dense(1, activation=tf.nn.sigmoid)(x)
    DeepSUMO = Model(inputs=[inputs1], outputs=[output], name="DeepSUMO")
    DeepSUMO.summary()
    DeepSUMO.compile(optimizer=optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'],
                  experimental_run_tf_function=False)
    return DeepSUMO